# Computer Vision Homework 1

Question 4

Amirkabir University of Technology

Dr. Safabakhsh

By Gholamreza Dar 400131018

Fall 2022


## Imports

In [ ]:
import cv2
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(); sns.set_style('dark')

import os
import datetime

## Functions

In [ ]:
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def disp(img, title='', s=8, vmin=None, vmax=None):
    plt.figure(figsize=(s,s))
    plt.axis('off')
    if vmin is not None and vmax is not None:
        plt.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.show()

def uint8ify(img):
    img -= img.min()
    img /= img.max()
    img *= 255
    return np.uint8(img)

def overlay(a,b):
    # a and b should be float images and between 0-1

    mask = a >= 0.5 # generate boolean mask of everywhere a > 0.5 
    ab = np.zeros_like(a) # generate an output container for the blended image 

    # now do the blending 
    ab[~mask] = (2*a*b)[~mask] # 2ab everywhere a<0.5
    ab[mask] = (1-2*(1-a)*(1-b))[mask] # else this
    
    return ab

def before_after(img_a, img_b, name='', vmin=None, vmax=None, effect_name='Processed', figsize=(10,4)):
    fig, axs = plt.subplots(1,2, constrained_layout=True, figsize=figsize)
    axs[0].axis('off')
    axs[1].axis('off')
    axs[0].set_title(f'{name} Original')
    axs[1].set_title(f'{name} {effect_name}')
    if vmin is not None and vmax is not None:
        axs[0].imshow(img_a, cmap='gray', vmin=vmin, vmax=vmax)
        axs[1].imshow(img_b, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        axs[0].imshow(img_a, cmap='gray')
        axs[1].imshow(img_b, cmap='gray')
    plt.show()

## CvtColor

In [ ]:
image = rgb(cv2.imread('data/edge.jpg'))

disp(image, 'edge.jpg', s=9)


## Find Edges using Canny

In [ ]:
## find edges using canny

image_edges1 = cv2.Canny(image, 0, 255)
# disp(image_edges1, 'Canny Edges | 0, 255', s=12)
before_after(image, image_edges1, name='Canny Edges', effect_name='| 0, 255', figsize=(10,10))

In [ ]:
## find edges using canny ( change thresholds )

image_edges2 = cv2.Canny(image, 180, 235)
# disp(image_edges2, 'Canny Edges | 180, 235', s=12)
before_after(image, image_edges2, name='Canny Edges', effect_name='| 180, 235', figsize=(10,10))

In [ ]:
## Blur and then find edges using canny

image_blur = cv2.GaussianBlur(image, (5,5), 0)
image_blur_edges = cv2.Canny(image_blur, 0, 255)
# disp(image_blur_edges, 'Blur + Canny Edges | 0, 255', s=12)
before_after(image, image_blur_edges, name='Blur + Canny Edges', effect_name='| 0, 255', figsize=(10,10))

In [ ]:
## Blur and then find edges using canny

image_blur = cv2.GaussianBlur(image, (3,3), 0)
image_blur_edges = cv2.Canny(image_blur, 100, 235)
# disp(image_blur_edges, 'Blur + Canny Edges | 100, 235', s=12)
before_after(image, image_blur_edges, name='Blur + Canny Edges | 100, 235', figsize=(10,10))

In [ ]:
## Blur and then find edges using canny
from ipywidgets import interact

@interact(kernel_size=(1,11), t1=(0,255), t2=(0,255))
def interactive_canny(kernel_size=3, t1=100, t2=235):
    if kernel_size %2==0:
        kernel_size += 1
    image_blur = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    image_blur_edges = cv2.Canny(image_blur, t1, t2)
    # disp(image_blur_edges, 'Blur + Canny Edges | 100, 235', s=12)
    before_after(image, image_blur_edges, name='Blur + Canny Edges | 100, 235', figsize=(10,10))

## Garbage Mask

In [ ]:
garbage_mask = np.zeros_like(image)
h, w = garbage_mask.shape[:2]
vertices = [
    [[
        [int(0.3*w), int(0.4*h)], 
        [int(0.3*w), int(0.6*h)], 
        [int(0.7*w), int(0.6*h)], 
        [int(0.7*w), int(0.4*h)]
    ]]
]
cv2.fillPoly(garbage_mask, np.array(vertices), (255, 255, 255))
garbage_mask = cv2.cvtColor(garbage_mask, cv2.COLOR_RGB2GRAY)
# weighted sum garbage mask and image
garbage_mask_preview = cv2.addWeighted(garbage_mask, 0.5, image[:,:,0], 0.5, 0)
before_after(image, garbage_mask_preview, name='"edge.jpg"', effect_name='Garbage Mask Preview', figsize=(10, 10))
disp(garbage_mask, 'Garbage Mask', s=12)

In [ ]:
# mask image_blur_edges with garbage_mask
masked_image_blur_edges = cv2.bitwise_and(image_blur_edges, garbage_mask)
# disp(masked_image_blur_edges, 'Blur + Canny Edges + Garbage Mask | 100, 235', s=12)
before_after(image, masked_image_blur_edges, name='"edge.jpg"', effect_name='Blur + Canny Edges + Garbage Mask | 100, 235', figsize=(10, 10))


In [ ]:
before_after(image_edges1, masked_image_blur_edges, name='Edge', effect_name='Blur + Canny Edges + Garbage Mask | 100, 235', figsize=(10, 10))